# DermaCon-IN: Swin Transformer Model Inferencing

This code is for infering the best model, Swin Transformer for Main_class prediction which has the below performance on validation data:

- ✅ Val Accuracy: 0.7088
- ✅ Precision: 0.7026
- ✅ Recall: 0.7088
- ✅ F1 Score: 0.7022
- ✅ Macro AUC: 0.7864
- Class 0 (Infectious Disorders): AUC = 0.8482
- Class 1 (Inflammatory Disorders): AUC = 0.8274
- Class 2 (Keratanisation Disorders): AUC = 0.6627
- Class 3 (Neoplasms and tumors): AUC = 0.8705
- Class 4 (No Definite Diagnosis): AUC = 0.5775
- Class 5 (Other skin disorders): AUC = 0.6357
- Class 6 (Pigmentary Disorders): AUC = 0.9110
- Class 7 (Skin Appendages Disorders): AUC = 0.9580

🎯 Per-Class Accuracy:
- Class 0 (Infectious Disorders): Accuracy = 0.7751
- Class 1 (Inflammatory Disorders): Accuracy = 0.6851
- Class 2 (Keratanisation Disorders): Accuracy = 0.1429
- Class 3 (Neoplasms and tumors): Accuracy = 0.1667
- Class 4 (No Definite Diagnosis): Accuracy = 0.0000
- Class 5 (Other skin disorders): Accuracy = 0.1429
- Class 6 (Pigmentary Disorders): Accuracy = 0.7317
- Class 7 (Skin Appendages Disorders): Accuracy = 0.8119

The below code loads the best checkpoint with an accuracy of . Then, infers this model on a sample image.

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import os
import numpy as np
from timm import create_model
import torch.nn.functional as F
import pandas as pd

# === CONFIGURATION ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint_path = "./checkpoints/Swin_MC_best_model.pth"  # replace if different
image_path = "./DATASET/IMG_0618.jpg"  # update with actual test image
csv_path = "./METADATA/test_split.csv"

# === LABEL MAPPING ===
df = pd.read_csv(csv_path)
unique_labels = sorted(df['Main_class'].dropna().unique().tolist())
label_to_idx = {lbl: i for i, lbl in enumerate(unique_labels)}
idx_to_label = {i: lbl for lbl, i in label_to_idx.items()}

# === TRANSFORM (same as validation) ===
mean = [0.53749797, 0.45875554, 0.40382471]
std = [0.21629889, 0.20366619, 0.20136241]
val_transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

# === LOAD MODEL ===
model = create_model(
    'swin_base_patch4_window12_384',
    pretrained=False,
    num_classes=len(label_to_idx),
    img_size=512
).to(device)

checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint)
model.eval()

# === LOAD & PROCESS IMAGE ===
img = Image.open(image_path).convert('RGB')
img_tensor = val_transform(img).unsqueeze(0).to(device)  # add batch dimension

# === INFERENCE ===
with torch.no_grad():
    output = model(img_tensor)             # (1, num_classes)
    probs = F.softmax(output, dim=1)       # softmax for probability
    pred_idx = probs.argmax(dim=1).item()  # predicted label index
    pred_label = idx_to_label[pred_idx]
    confidence = probs[0, pred_idx].item()

# === RESULT ===
print(f"✅ Predicted Class: {pred_label} (index: {pred_idx})")
print(f"🔢 Confidence Score: {confidence:.4f}")


/tmp/ipykernel_1128741/2051366583.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


✅ Predicted Class: Keratanisation Disorders (index: 2)
🔢 Confidence Score: 0.9216
